# ACSE-3 (Numerical Methods) <a class="tocSkip">

## Lecture 9: Partial Differential Equations (PDEs) 2 <a class="tocSkip">
    
### Homework exercises <a class="tocSkip">

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Homework" data-toc-modified-id="Homework-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Homework</a></span><ul class="toc-item"><li><span><a href="#Homework---Comparing-linear-solvers-(Jacobi-vs-Gauss-Seidel)" data-toc-modified-id="Homework---Comparing-linear-solvers-(Jacobi-vs-Gauss-Seidel)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Homework - Comparing linear solvers (Jacobi vs Gauss-Seidel)</a></span></li><li><span><a href="#Homework---matrix-based-solvers-[$\star\star$---not-really-any-question-here,-just-read-through-solution-for-interest]" data-toc-modified-id="Homework---matrix-based-solvers-[$\star\star$---not-really-any-question-here,-just-read-through-solution-for-interest]-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Homework - matrix based solvers [$\star\star$ - not really any question here, just read through solution for interest]</a></span><ul class="toc-item"><li><span><a href="#Imposing-boundary-conditions" data-toc-modified-id="Imposing-boundary-conditions-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Imposing boundary conditions</a></span></li></ul></li><li><span><a href="#Homework---A-case-with-a-non-zero-RHS-and-homogeneous-Dirichlet-BCs" data-toc-modified-id="Homework---A-case-with-a-non-zero-RHS-and-homogeneous-Dirichlet-BCs-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Homework - A case with a non-zero RHS and homogeneous Dirichlet BCs</a></span></li><li><span><a href="#Homework---A-case-with-a-non-zero-RHS-and-inhomogeneous-Dirichlet-BCs" data-toc-modified-id="Homework---A-case-with-a-non-zero-RHS-and-inhomogeneous-Dirichlet-BCs-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Homework - A case with a non-zero RHS and inhomogeneous Dirichlet BCs</a></span></li><li><span><a href="#Homework---Alternative-solver-methods-for-Navier-Stokes-[$\star\star\star$]" data-toc-modified-id="Homework---Alternative-solver-methods-for-Navier-Stokes-[$\star\star\star$]-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Homework - Alternative solver methods for Navier-Stokes [$\star\star\star$]</a></span></li></ul></li></ul></div>

In [1]:
%precision 3
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as sl
import scipy.sparse as sp
import scipy.sparse.linalg as spla
# the following allows us to plot triangles indicating convergence order
from mpltools import annotation
# as we're in 2D we will be doing some 3D plotting
from mpl_toolkits.mplot3d import Axes3D
# and using some colormaps
from matplotlib import cm
# and we will create some animations!
import matplotlib.animation as animation
from IPython.display import HTML
from pprint import pprint

# Homework


## Homework - Comparing linear solvers (Jacobi vs Gauss-Seidel)

In the lecture we considered the problem 

$$ \frac{\partial^2 c}{\partial x^2} + \frac{\partial^2 c}{\partial y^2} \equiv \nabla^2 c = f, $$

with an analytical solution which was chosen such that its Laplacian is zero (i.e. $f\equiv 0$)

$$ c_{\text{exact}}(x,y) =\frac{\sin(2\pi x)\sinh(2\pi y)}{\sinh(2\pi)},$$

and we used the value of this exact solution to define Dirichlet BCs for our problem.

The difference between the Jacobi solver we used in the lecture and Gauss-Seidel is that with G-S we make use of updated solution values as soon as they are available - we don't wait until the next iteration to make use of them. We saw in L3 that this approach can lead to faster convergence (in terms of the number of  required iterations - of course not overall if each iteration costs a different amount).

We can write the iterative scheme in the Gauss-Seidel case as:

$$ c^{n+1}_{i,j} = \frac{1}{4}\left(c^{n}_{i+1,j}  + c_{i-1,j}^{n+1} + c^{n}_{i,j+1} + c^{n+1}_{i,j-1} - \Delta x^2 f_{i,j} \right), $$

where we have made an assumption on the order we perform the update in $i$ and $j$, i.e. for a given $i,j$ we assume we have already visiting and updated the values at the $i-1$ and the $j-1$ locations.

Write some code to compare the number of iterations and overall time to solve the above Poisson problem with Dirichlet BCs  to a given tolerance using our vectorised Jacobi code from the lecture, as well as non-vectorised versions of Jacobi and Gauss-Seidel. 

In order to compute timings you could turn these three solvers into functions and use `%timeit` as we have done in previous homeworks.

## Homework - matrix based solvers [$\star\star$ - not really any question here, just read through solution for interest]

Alternatively (to open up a wider range of solver options) we can cast the problem is the matrix form

$$A\boldsymbol{C} = \boldsymbol{b},$$

and solve with standard methods such as Gaussian elimination or conjugate gradients (we will see that for the Laplace operator $A$ is symmetric positive definite).

The first step for constructing the matrix-vector multiplication on the LHS is to assume a numbering which allows us to reshape the $N_x\times N_y$ unknowns making up 

$$\boldsymbol{c} = \{c_{ij}\}, \;\;\;\;\; i=0,\ldots, N_x-1,\;\;\;\; j=0,\ldots, N_y-1,$$  

into an $N_xN_y \times 1$ column vector 

$$\boldsymbol{C} = \{C_{k}\}, \;\;\;\;\; k=0,\ldots, N_x\times N_y-1.$$

There are several ways we can go back and forth between these numberings in Python - read the docs on `reshape`, `flatten`, `ravel`:

(note that these implement what is called [*row major ordering*](https://en.wikipedia.org/wiki/Row-_and_column-major_order))

# So notice that exploiting the sparsity of the matrix is vitally important for overall algorithm efficiency.

In [6]:
a = np.array([[1,2,3],[4,5,6]])
pprint(a)
pprint(a.ravel())
pprint(a.flatten())
pprint(a.reshape(-1))

print('\n')
# this is how we can go back again
a2 = a.ravel()
pprint(a2)
pprint(a2.reshape(np.shape(a)))

print('\n')
# this shows that "C ordering" is the default row-major behaviour we will use
pprint(a.ravel(order='C'))
# and this shows that "Fortran ordering" is column-major
pprint(a.ravel(order='F'))

array([[1, 2, 3],
       [4, 5, 6]])
array([1, 2, 3, 4, 5, 6])
array([1, 2, 3, 4, 5, 6])
array([1, 2, 3, 4, 5, 6])


array([1, 2, 3, 4, 5, 6])
array([[1, 2, 3],
       [4, 5, 6]])


array([1, 2, 3, 4, 5, 6])
array([1, 4, 2, 5, 3, 6])


### Imposing boundary conditions


1. The simplest approach to applying Dirichlet BCs would be to replace the corresponding row of the matrix with a 1 on the diagonal, zero in all other row entries and set the Dirichlet BC value in the corresponding row of the RHS matrix. The problem with this approach is that it destroys the symmetry of the LHS matrix which we can exploit in solvers (and indeed which is a requirement for the use of certain solvers) - this is because we have edited the row, but can't similarly edit the corresponding column without messing up the discretisation for other nodes.


2. The second approach is to leave the row unchanged but set a very large number on the diagonal, and set this large number multiplied by our Dirichlet BC value in the corresponding row of the RHS matrix. This is sometimes called the *big spring* method.


3. And finally *Lifting*, which we do below, which involves removing the rows and columns corresponding to the boundary nodes from the discretisation matrix.

## Homework - A case with a non-zero RHS and homogeneous Dirichlet BCs

Let's consider now a Poisson problem with a non-zero RHS and zero Dirichlet BCs.

Our PDEs is
$$ \frac{\partial^2 c}{\partial x^2} + \frac{\partial^2 c}{\partial y^2} \equiv \nabla^2 c = f, $$

with exact solution that is constructed to take the value zero on all boundaries:

$$ c_{\text{exact}}(x,y) = x^2 (1 - x)\sin(\pi y). $$

We can easily evaluate the Laplacian of this function to give us the RHS function:

$$ f = \sin(\pi y)  ( 2 - 6 x - (\pi x)^2  (1 - x) ). $$

Solve this problem using one of your solvers from above.

## Homework - A case with a non-zero RHS and inhomogeneous Dirichlet BCs

Consider now a case with

$$ c_{\text{exact}}(x,y) = \exp\left(x + \frac{y}{2}\right), $$

which is non-zero on the boundaries, and hence this same function can be used to specify Dirichlet BCS.

We can easily evaluate the Laplacian of this function to give us the RHS function:

$$ f = \frac{5}{4}\exp\left(x + \frac{y}{2}\right). $$

Solve this problem using one of your solvers.

## Homework - Alternative solver methods for Navier-Stokes [$\star\star\star$]

In the lecture a major cost of our Navier-Stokes solver is the solution of the pressure Poisson equation within the pressure projection method.

Try updating our CFD solver from lecture to use an LU based matrix solver for pressure instead.

You will need to consider several things here:


1. The imposition of homogeneous Neumann BCs in the generation of the discretisation matrix $A$.


2. The imposition of the pressure level node in the generation of the discretisation matrix $A$ (I suggest for simplicity you use the big spring method here).

<br>

**Important comments on performance optimisation** 

Now while simple our Jacobi solver from the lecture actually performs quite well for our lid driven cavity case. Firstly, as considered above, as the solve is fully vectorised that means that each iteration is very efficient. Secondly, as we are solving this problem to steady state we need to do a lot of time step where things don't change much from one to the next. This is an advantage for the iterative solvers utilised within each time step: starting the Jacobi iteration using an initial guess from the previous time level means that we can minimise the number of iterations required to converge. Indeed for the lid-driven cavity we observe that while for early time levels we have to use a large number of iterations, for much of latter parts of the solve Jacobi only requires a single iteration to converge, this is because the pressure is not changing by very much between time levels.  **For a fully time-dependent problem a simple solver like Jacobi is not viable**, the ability to start from a reasonable initial guess (from the previous time level) and simply the overall size of the problem, points to an optimal solution being a **matrix based iterative solver such a preconditioned CG**.